In [49]:
import os
import h5py

import cv2
import numpy

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [50]:
NAME_CROP = "crop_train.h5"
NAME_TEST = "test.h5"

In [51]:
DATA_PATH = "./image/"
DATA_FORMAT = ".png"

Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6
scale = 2

In [52]:
def prepare_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = numpy.zeros((nums * Random_Crop, 1, Patch_size, Patch_size), dtype=numpy.double)
    label = numpy.zeros((nums * Random_Crop, 1, label_size, label_size), dtype=numpy.double)

    for i in range(2):
        if DATA_FORMAT in names[i]:
            name = _path + names[i]

            hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
            #hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2RGB)
            shape = hr_img.shape

            hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
            hr_img = hr_img[:, :, 0]

            lr_img = cv2.resize(hr_img, (int(shape[1] / scale), int(shape[0] / scale)))
            lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

            Points_x = numpy.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)
            Points_y = numpy.random.randint(0, min(shape[0], shape[1]) - Patch_size, Random_Crop)

            for j in range(Random_Crop):
                lr_patch = lr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]
                hr_patch = hr_img[Points_x[j]: Points_x[j] + Patch_size, Points_y[j]: Points_y[j] + Patch_size]

                lr_patch = lr_patch.astype(float) / 255.
                hr_patch = hr_patch.astype(float) / 255.

                data[i * Random_Crop + j, 0, :, :] = lr_patch
                label[i * Random_Crop + j, 0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

                #plt.figure(figsize=(16,16))
                #plt.subplot(1,2,1)
                #plt.title('lr')
                #plt.imshow(lr_patch)
                #plt.subplot(1,2,2)
                #plt.title('hr')
                #plt.imshow(hr_patch)
    
    return data, label

In [53]:
# BORDER_CUT = 8
BLOCK_STEP = 16
BLOCK_SIZE = 32

In [54]:
def prepare_crop_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = []
    label = []

    for i in range(nums):
        if DATA_FORMAT in names[i]:
            name = _path + names[i]
            hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
            hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
            hr_img = hr_img[:, :, 0]
            shape = hr_img.shape

            lr_img = cv2.resize(hr_img, (int(shape[1] / scale), int(shape[0] / scale)))
            lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

            width_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
            height_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
            for k in range(int(width_num)):
                for j in range(int(height_num)):
                    x = k * BLOCK_STEP
                    y = j * BLOCK_STEP
                    hr_patch = hr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                    lr_patch = lr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]

                    lr_patch = lr_patch.astype(float) / 255.
                    hr_patch = hr_patch.astype(float) / 255.

                    lr = numpy.zeros((1, Patch_size, Patch_size), dtype=numpy.double)
                    hr = numpy.zeros((1, label_size, label_size), dtype=numpy.double)

                    lr[0, :, :] = lr_patch
                    hr[0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

                    data.append(lr)
                    label.append(hr)

    data = numpy.array(data, dtype=float)
    label = numpy.array(label, dtype=float)
    return data, label

In [55]:
def write_hdf5(data, labels, output_filename):
    """
    This function is used to save image data and its label(s) to hdf5 file.
    output_file.h5,contain data and label
    """

    x = data.astype(numpy.float32)
    y = labels.astype(numpy.float32)

    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
        # h.create_dataset()

In [56]:
data, label = prepare_crop_data(DATA_PATH)
write_hdf5(data, label, NAME_CROP)
print(NAME_CROP + " generated")
data, label = prepare_data(DATA_PATH)
write_hdf5(data, label, NAME_TEST)
print(NAME_TEST + " generated")

crop_train.h5 generated
test.h5 generated
